# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine

import sglang as sgl
import asyncio

llm = sgl.Engine(model_path="meta-llama/Meta-Llama-3.1-8B-Instruct")

Loading safetensors checkpoint shards:   0% Completed | 0/4 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:  25% Completed | 1/4 [00:00<00:02,  1.27it/s]


Loading safetensors checkpoint shards:  50% Completed | 2/4 [00:01<00:01,  1.19it/s]


Loading safetensors checkpoint shards:  75% Completed | 3/4 [00:02<00:00,  1.14it/s]


Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.56it/s]
Loading safetensors checkpoint shards: 100% Completed | 4/4 [00:02<00:00,  1.39it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

...

 70%|██████▉   | 16/23 [00:04<00:01,  4.86it/s]

 78%|███████▊  | 18/23 [00:05<00:00,  5.27it/s]

 87%|████████▋ | 20/23 [00:05<00:00,  5.47it/s]

 96%|█████████▌| 22/23 [00:05<00:00,  5.50it/s]

100%|██████████| 23/23 [00:06<00:00,  3.76it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Alvin, and I'm a programmer with a passion for building innovative web applications. I'm currently working on a project that involves creating a web scraper to extract data from a website. The website in question has a complex structure, with multiple nested tables and a lot of JavaScript-generated content.

To tackle this challenge, I've decided to use Selenium WebDriver for browser automation, as it allows me to render the website in a real browser and execute JavaScript code. However, I'm running into some issues with the website's layout and JavaScript-generated content.

Here's a simplified example of the HTML structure:
```html
<div class="main-container">

Prompt: The president of the United States is
Generated text:  not the same as the president of the United States government. The United States government is a sovereign state that governs the country, while the president is the head of that government. A president is a political lead

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing synchronous streaming generation ===")

for prompt in prompts:
    print(f"\nPrompt: {prompt}")
    print("Generated text: ", end="", flush=True)

    for chunk in llm.generate(prompt, sampling_params, stream=True):
        print(chunk["text"], end="", flush=True)
    print()


=== Testing synchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Karen

,

 and

 I

...

,

 reducing

 latency

 and

 improving

### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Hello, my name is
Generated text:  Steve. I am a 56-year-old father of two grown children, living in a small town in upstate New York. I have been married to my wonderful wife, Mary, for 34 years. I work as a printer for a small business in town. I enjoy reading, hiking, and gardening in my free time.
I also enjoy photography, and I have been taking pictures for over 40 years. I have even had a few of my photos published in local magazines and newspapers. I love capturing the beauty of the natural world, and I am always looking for new and interesting subjects to photograph.
I am a bit of

Prompt: The capital of France is
Generated text:  a city of grandeur, beauty and elegance. Paris has been a center of culture, fashion, and history for centuries and attracts millions of visitors every year. From the Eiffel Tower to the Louvre, the city is a treasure trove of iconic landmarks and artistic treasures. Here are some of the top things to do in Paris.
The Eiffel Tower is a must-v

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Hello, my name is",
    "The capital of France is",
    "The future of AI is",
]
sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        generator = await llm.async_generate(prompt, sampling_params, stream=True)
        async for chunk in generator:
            print(chunk["text"], end="", flush=True)
        print()


asyncio.run(main())


=== Testing asynchronous streaming generation ===

Prompt: Hello, my name is
Generated text: 

 Holly

!

 I

 am

...

 understanding

 of

 user

 intent

.


In [6]:
llm.shutdown()